In [1]:
from bokeh.models import ColumnDataSource, Slider, CustomJS, HoverTool, LegendItem, Legend,  CDSView, GroupFilter
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.tile_providers import get_provider, Vendors
from bokeh.transform import linear_cmap
from bokeh.models.widgets import CheckboxGroup

output_notebook()

prost_df['Year'] = pd.to_datetime(df['Date']).dt.year

# Aggregate incidents by year and address, ensure PdDistrict is included in the aggregation
agg_df = prost_df.groupby(['Address', 'Year', 'PdDistrict']).agg({
    'X': 'first',  # Assume X is consistent per address, take the first occurrence
    'Y': 'first'  # Same assumption for Y
}).reset_index()

# After resetting the index, add a 'Counts' column by counting the size of each group
agg_df['Counts'] = prost_df.groupby(['Address', 'Year', 'PdDistrict']).size().reset_index(name='Counts')['Counts']


# Convert lat/lon to Web Mercator here if necessary
agg_df[['mercator_x', 'mercator_y']] = agg_df.apply(
    lambda row: x_y_to_web_mercator(row['X'], row['Y']), axis=1, result_type="expand"
)

### TYPES OF PROSTITUTION --- ADD HERE!
focusdesc = ['SOLICITS FOR ACT OF PROSTITUTION', 'SOLICITS TO VISIT HOUSE OF PROSTITUTION', 'LOITERING FOR PURPOSE OF PROSTITUTION', 'HUMAN TRAFFICKING']
filtered_df = prost_df[prost_df["Descript"].isin(focusdesc)]
grouped = filtered_df.groupby(["Address", "Year", "Descript"]).size().reset_index(name='Counts')
pivot_table = grouped.pivot_table(index=["Address", "Year"], columns="Descript", values="Counts", fill_value=0).reset_index()

# Merge with agg_df (which should already have "Address" and "Year" columns)
agg_df_with_types = pd.merge(agg_df, pivot_table, on=["Address", "Year"], how="left").fillna(0)

# Ensure integer types for counts after merge and fillna
type_columns = ['SOLICITS FOR ACT OF PROSTITUTION', 'SOLICITS TO VISIT HOUSE OF PROSTITUTION', 'LOITERING FOR PURPOSE OF PROSTITUTION', 'HUMAN TRAFFICKING']
agg_df_with_types[type_columns] = agg_df_with_types[type_columns].astype(int)

agg_df = agg_df_with_types


### --- END TYPES


## --- Circle Sizes Normalized --- 
# Assuming 'Counts' are the number of incidents
min_size = 5  # Minimum circle size
max_size = 80  # Maximum circle size

# Normalize 'Counts' to a scale of min_size to max_size
min_count = agg_df['Counts'].min()
max_count = agg_df['Counts'].max()

# Calculate normalized sizes
agg_df['normalized_size'] = min_size + (agg_df['Counts'] - min_count) / (max_count - min_count) * (max_size - min_size)

# Ensure the 'normalized_size' doesn't exceed max_size
agg_df['normalized_size'] = agg_df['normalized_size'].clip(upper=max_size)

initial_year_data = agg_df[agg_df['Year'] == 2003]

source = ColumnDataSource(data=initial_year_data)


## --- Center the map ----
average_longitude = -122.42493579160154
average_latitude = 37.77664605675238

# Assuming x_y_to_web_mercator is already defined as before
center_x, center_y = x_y_to_web_mercator(average_longitude, average_latitude)

# Define initial view bounds
x_range = (center_x - 4000, center_x + 2000)  # Adjust as needed
y_range = (center_y - 9000, center_y + 6000)  # Adjust as needed




## --- Create Bokah map --- 

# Initialize figure
#p = figure(x_range=x_range, y_range=y_range,
p = figure(x_range=x_range, y_range=y_range,
            x_axis_type="mercator", y_axis_type="mercator",
            width=1200, height=600, title="San Francisco Prostitution Incidents")


# Directly use the tile provider
p.add_tile(CARTODBPOSITRON)

#palette="Viridis256"
color_mapper = LinearColorMapper(palette="Inferno256", low=min_count, high=max_count)


# Add hover tool to display counts and types
hover = HoverTool()
hover.tooltips = [
    ("Address", "@Address"),
    ("Total Incidents", "@Counts"),
    ("Solicits for Act of Prostitution", "@{SOLICITS FOR ACT OF PROSTITUTION}"),
    ("Solicits to Visit House of Prostitution", "@{SOLICITS TO VISIT HOUSE OF PROSTITUTION}"),
    ("Loitering for Purpose of Prostitution", "@{LOITERING FOR PURPOSE OF PROSTITUTION}"),
    ("Human Trafficking", "@{HUMAN TRAFFICKING}")
]

p.add_tools(hover)



#----- NYT START ----
# Create separate glyphs for each type of prostitution
glyphs = {}
for descript in focusdesc:
    view = CDSView(filters=[GroupFilter(column_name='Descript', group=descript)])
    circle = p.circle(x='mercator_x', y='mercator_y', source=source, view=view, size='normalized_size', 
                      color=linear_cmap('Counts', "Inferno256", min_count, max_count),
                      fill_alpha=0.6, line_color=None, legend_label=descript)
    glyphs[descript] = circle

# Setup legend to allow for muting
p.legend.click_policy="mute"
p.add_layout(p.legend[0], 'left')  # This adds the legend outside the plot

#----- NYT END ----


### ----- SLIDER
slider = Slider(start=2003, end=2017, value=2003, step=1, title="Year", width=300, align="center")

# CustomJS callback for Slider
callback = CustomJS(args=dict(source=source, original_data=agg_df.to_dict(orient='list'), slider=slider), code="""
    const year = slider.value;
    const newData = {Address: [], Year: [], PdDistrict: [], X: [], Y: [], Counts: [], mercator_x: [], mercator_y: [], normalized_size: [], 'SOLICITS FOR ACT OF PROSTITUTION': [], 'SOLICITS TO VISIT HOUSE OF PROSTITUTION': [], 'LOITERING FOR PURPOSE OF PROSTITUTION': [], 'HUMAN TRAFFICKING': []};
    
    for (let i = 0; i < original_data.Year.length; i++) {
        if (original_data.Year[i] === year) {
            newData.Address.push(original_data.Address[i]);
            newData.Year.push(original_data.Year[i]);
            newData.PdDistrict.push(original_data.PdDistrict[i]);
            newData.X.push(original_data.X[i]);
            newData.Y.push(original_data.Y[i]);
            newData.Counts.push(original_data.Counts[i]);
            newData.mercator_x.push(original_data.mercator_x[i]);
            newData.mercator_y.push(original_data.mercator_y[i]);
            newData.normalized_size.push(original_data.normalized_size[i]);
            newData['SOLICITS FOR ACT OF PROSTITUTION'].push(original_data['SOLICITS FOR ACT OF PROSTITUTION'][i]);
            newData['SOLICITS TO VISIT HOUSE OF PROSTITUTION'].push(original_data['SOLICITS TO VISIT HOUSE OF PROSTITUTION'][i]);
            newData['LOITERING FOR PURPOSE OF PROSTITUTION'].push(original_data['LOITERING FOR PURPOSE OF PROSTITUTION'][i]);
            newData['HUMAN TRAFFICKING'].push(original_data['HUMAN TRAFFICKING'][i]);
        }
    }
    source.data = newData;
    
""")

slider.js_on_change('value', callback)
### ------ SLIDER DONE






# Plot intersections as bubbles
p.circle(x='mercator_x', y='mercator_y', source=source, size='normalized_size', 
         fill_color={'field': 'Counts', 'transform': color_mapper},
         fill_alpha=0.6, line_color=None)


# Optional: Update color bar to reflect actual counts
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title="Counts")
p.add_layout(color_bar, 'right')

# Layout and add to document

layout = column(slider, p, sizing_mode="scale_width")

show(layout)

#output_file(filename="prostitution_test.html")
#save(layout)

Loading BokehJS ...

NameError: name 'pd' is not defined